## 참고
- 본 코드는 빅데이터분석기사 실기 작업형 2유형을 위한 연습용 코드임

In [ ]:
Main_name = 'kingcountyprice'

trainData  = f'https://raw.githubusercontent.com/Datamanim/datarepo/main/{Main_name}/train.csv'
testData  = f'https://raw.githubusercontent.com/Datamanim/datarepo/main/{Main_name}/test.csv'
subData  = f'https://raw.githubusercontent.com/Datamanim/datarepo/main/{Main_name}/submission.csv'

import warnings
warnings.filterwarnings('ignore')
#데이터로드
import pandas as pd

train = pd.read_csv(trainData)
test = pd.read_csv(testData)
submission = pd.read_csv(subData)


drop_col = ['id','date','zipcode','lat','long']

train.drop(drop_col,axis=1,inplace=True)
test.drop(drop_col,axis=1,inplace=True)

y = train['price']
X = train.iloc[:,1:]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size = 0.3,random_state = 42 )

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

sc_mm = MinMaxScaler()
sc_ss = StandardScaler()

# 1. MInMax로 했을 때
X_train_mm = sc_mm.fit_transform(X_train)
X_test_mm = sc_mm.transform(X_test)

## dir(sklearn.ensemble) ##
import sklearn

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor( random_state= 42 )      # parameter : n_estimators = 100, max_depth = 3 
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)

from sklearn.metrics import r2_score
print('RF r2_score: ',r2_score(y_test, rf_y_pred)) # RF r2_score:  0.7284506460848885


# GradientBoostingRegressor / help(sklearn.ensemble.GradientBoostingRegressor)
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state= 42)     # learning_rate(0.1 ~ 0,01 ), n_estimators(100 ), max_depth(defalut : 3), random_state , verbose(0 or 1)
gbr.fit(X_train, y_train)
gbr_y_pred = gbr.predict(X_test)
print('gbr r2 score: ', r2_score(y_test, gbr_y_pred)) # 0.7171536002097245

## dir(sklearn.linear_model) ##
from sklearn.linear_model import LinearRegression
lrr = LinearRegression()
lrr.fit(X_train, y_train)
lrr_y_pred =lrr.predict(X_test) 
print('Linear Regression r2 score: ', r2_score(y_test, lrr_y_pred)) #  0.6545932706051483

# Ridge
from sklearn.linear_model import Ridge, Lasso
ridge = Ridge()
ridge.fit(X_train, y_train)
ridge_y_pred = ridge.predict(X_test)
print('릿지 r2 score ', r2_score(y_test, ridge_y_pred)) # 0.6547560311779191

# Lasso
lasso = Lasso() # max_iter ,alpha
lasso.fit(X_train, y_train)
lasso_y_pred = lasso.predict(X_test)
print('라쏘 r2 score', r2_score(y_test, lasso_y_pred)) #  0.6545965041603428


## dir(xgboost) ##
# xgboost - XGBRegressor
from xgboost import XGBRFRegressor 
xgbrf = XGBRFRegressor(random_state =42)
xgbrf.fit(X_train, y_train)
xgbrf_y_pred = xgbrf.predict(X_test)
print('xgbRF r2 score ', r2_score(y_test, xgbrf_y_pred)) # 0.5995687211119499

# xgboost - XGBRegressor 
from xgboost import XGBRegressor
xgb= XGBRegressor(random_state=42)# max_depth, learning_rate, n_estimators
xgb.fit(X_train, y_train) 
xgb_y_pred = xgb.predict(X_test) 
print('xgb_reg r2 score', r2_score(y_test, xgb_y_pred)) 0.7176627781396154


submission = rf.predict(test)



## 그리드서치 수행

### 그리드 서치 전

In [ ]:
## # n_estimators 는 tree의 개수를 의미  
## RandomForestRegressor 
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor( random_state= 42 )      # parameter : n_estimators = 100, max_depth = 3 
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

# 결과 살펴보기 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error 
import numpy as np 

print('MAE:', mean_absolute_error(y_test, y_pred)) 

print('MSE:', mean_squared_error(y_test, y_pred)) 

print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

print('R2:', r2_score(y_test,y_pred))

### 그리드서치 후

In [ ]:
param_grid = [ {'n_estimators': [50,100,200], 'max_features':[2, 4, 6, 8]}, {'bootstrap': [False]}] 

RF_reg2 = RandomForestRegressor() 

grid_search = GridSearchCV(RF_reg2, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train) 


print('grid_search.best_params_', grid_search.best_params_) # 8 ,n_estimators : 200


In [ ]:
n_estimators = 200

max_features = 8

RF_reg = RandomForestRegressor(n_estimators=n_estimators, random_state=42, max_features=max_features) 
RF_reg.fit(X_train, y_train)
y_pred = RF_reg.predict(X_test)

print('## RandomForestRegressor > n_estimators:', n_estimators) 

print('MAE:', mean_absolute_error(y_test, y_pred)) 

print('MSE:', mean_squared_error(y_test, y_pred)) 

print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

print('R2 score ', r2_score(y_test, y_pred)) ## 0.72 -> 0.73 .. ...?